## Search the Best Arbitrage in Bostrom Pools


In [6]:
import pandas as pd
from IPython.core.display import display, HTML

from src.data_extractors import get_pools, get_prices

import os


MIN_ARBITRAGE_REVENUE = 0.01
H_START_AMOUNT = 3_500_000_000


/var/folders/bp/n70b2l816310jdkr3rqn11t00000gn/T/ipykernel_70426/4263856539.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
path = "/Users/user/Documents/local_files/cyber_files/"
os.chdir(path)

#### Get Bostrom and Osmosis Pools Data


In [8]:
pools_df = get_pools(display_data=True)


network,id,type_id,balances,swap_fee,reserve_coin_denoms
bostrom,1,1,"[{'denom': 'boot', 'amount': '1117077383490'}, ...",0.003,"[boot, hydrogen]"
bostrom,2,1,"[{'denom': 'boot', 'amount': '13670799018'}, {'...",0.003,"[boot, milliampere]"
bostrom,3,1,"[{'denom': 'boot', 'amount': '1136349598958'}, ...",0.003,"[boot, tocyb]"
bostrom,4,1,"[{'denom': 'hydrogen', 'amount': '2018832997561...",0.003,"[hydrogen, tocyb]"
bostrom,5,1,"[{'denom': 'hydrogen', 'amount': '270643733363'...",0.003,"[hydrogen, milliampere]"
bostrom,6,1,"[{'denom': 'hydrogen', 'amount': '421439789534'...",0.003,"[hydrogen, millivolt]"
bostrom,7,1,"[{'denom': 'hydrogen', 'amount': '153006398874'...",0.003,"[hydrogen, uosmo in bostrom]"
bostrom,8,1,"[{'denom': 'hydrogen', 'amount': '21344327094'}...",0.003,"[hydrogen, uatom in bostrom]"
bostrom,9,1,"[{'denom': 'uosmo in bostrom', 'amount': '15663...",0.003,"[uosmo in bostrom, uatom in bostrom]"
bostrom,10,1,"[{'denom': 'milliampere', 'amount': '25348798'}...",0.003,"[milliampere, millivolt]"


In [9]:
#For excel spreadsheet

import datetime

_list = pools_df["balances"].to_list()
_t = []
for i in _list:
    _t.append([i[0]["denom"], i[0]["amount"], i[1]["denom"], i[1]["amount"]])

_pools = pd.DataFrame.from_records(
    _t,
    columns=["Coin 1", "Coin 1 pool amount", "Coin 2", "Coin 2 pool amount"],
)

_pools['update_time'] = datetime.datetime.now()

_pools.to_csv("csv/pools_log.csv" , mode='a', index=False, header=False)

In [113]:
# хочу понять глубину пулов
df = pd.DataFrame()
l = []
for i in pools_df["balances"]:
    p0 = {i[0]["denom"]: i[0]["amount"], i[1]["denom"]: i[1]["amount"]}
    l.append(p0)

df.from_records(l)


,boot,hydrogen,milliampere,tocyb,millivolt,uosmo in bostrom,uatom in bostrom,uatom in osmosis,uosmo,boot in osmosis
0,1126705265775,1953892685203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13321216168,NaN,4202131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1133358432482,NaN,NaN,807427174272,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1999821072942,NaN,823177443560,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,271420119663,46130150,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,427969035480,NaN,NaN,6362455,NaN,NaN,NaN,NaN,NaN
6,NaN,171263829916,NaN,NaN,NaN,1460532257,NaN,NaN,NaN,NaN
7,NaN,23573369724,NaN,NaN,NaN,NaN,23353529,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,1545628458,181481871,NaN,NaN,NaN
9,NaN,NaN,25382957,NaN,2205746,NaN,NaN,NaN,NaN,NaN


#### Calculate Prices


In [114]:
price_df = get_prices(pools_df=pools_df, display_data=True)


,uatom in osmosis,milliampere,uosmo in bostrom,uatom in bostrom,boot in osmosis,millivolt,boot,tocyb,hydrogen,uosmo
uatom in osmosis,1,NaN,NaN,1,0.00,NaN,NaN,NaN,NaN,0.12
milliampere,NaN,1,NaN,NaN,NaN,11.47,0.00,NaN,0.00,NaN
uosmo in bostrom,NaN,NaN,1,8.49,NaN,NaN,NaN,NaN,0.01,1
uatom in bostrom,1,NaN,0.12,1,NaN,NaN,NaN,NaN,0.00,NaN
boot in osmosis,400.20,NaN,NaN,NaN,1,NaN,1,NaN,NaN,47.21
millivolt,NaN,0.09,NaN,NaN,NaN,1,0.00,NaN,0.00,NaN
boot,NaN,"3,160.60",NaN,NaN,1,"38,200.44",1,1.40,0.57,NaN
tocyb,NaN,NaN,NaN,NaN,NaN,NaN,0.71,1,0.41,NaN
hydrogen,NaN,"5,866.14",116.91,"1,006.39",NaN,"67,062.97",1.73,2.42,1,NaN
uosmo,8.47,NaN,1,NaN,0.02,NaN,NaN,NaN,NaN,1


#### Search the Best Arbitrage


In [115]:
coins_unique_set = set(price_df.columns)
result_list = []
for coin_1 in coins_unique_set:
    coin_1_amount = H_START_AMOUNT * price_df.loc[coin_1, "hydrogen"]
    if coin_1_amount > 0:
        for coin_2 in coins_unique_set.difference({coin_1}):
            coin_2_amount = coin_1_amount * price_df.loc[coin_2, coin_1]
            if coin_2_amount > 0:
                for coin_3 in coins_unique_set.difference({coin_1, coin_2}):
                    coin_3_amount = coin_2_amount * price_df.loc[coin_3, coin_2]
                    result = coin_3_amount * price_df.loc[coin_1, coin_3]
                    if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                        result_list.append(
                            [[coin_1, coin_2, coin_3], round(result / coin_1_amount, 3)]
                        )
                    if coin_2_amount > 0:
                        for coin_4 in coins_unique_set.difference(
                            {coin_1, coin_2, coin_3}
                        ):
                            coin_4_amount = coin_3_amount * price_df.loc[coin_4, coin_3]
                            result = coin_4_amount * price_df.loc[coin_1, coin_4]
                            if result > coin_1_amount * (1 + MIN_ARBITRAGE_REVENUE):
                                result_list.append(
                                    [
                                        [coin_1, coin_2, coin_3, coin_4],
                                        round(result / coin_1_amount, 3),
                                    ]
                                )

result_df = pd.DataFrame(
    data=[
        [" -> ".join(coin for coin in item[0] + [item[0][0]]), item[1]]
        for item in result_list
    ],
    columns=["arbitrage", "revenue"],
).sort_values("revenue", ascending=False)

print(coin_1_amount)

pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{0:7,.2f}".format
display(HTML(result_df.to_html(index=False, notebook=True, show_dimensions=False)))


nan


arbitrage,revenue
boot -> milliampere -> hydrogen -> boot,1.06
hydrogen -> boot -> milliampere -> hydrogen,1.06
milliampere -> hydrogen -> boot -> milliampere,1.06
tocyb -> boot -> milliampere -> hydrogen -> tocyb,1.06
hydrogen -> tocyb -> boot -> milliampere -> hydrogen,1.06
milliampere -> hydrogen -> tocyb -> boot -> milliampere,1.06
boot -> milliampere -> hydrogen -> tocyb -> boot,1.06
boot -> milliampere -> millivolt -> hydrogen -> boot,1.05
millivolt -> hydrogen -> boot -> milliampere -> millivolt,1.05
hydrogen -> boot -> milliampere -> millivolt -> hydrogen,1.05


In [ ]:
# Хочу понять сколько нужно поменять чего для эквилибриума
# Я меняю 
# z
